<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_Git%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GitHub参照版

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます

>～使い方～  
>①左上　PlayBackgroundを選択  
>②左端の▶を押して実行

2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/06 試作　GitHub版  

In [0]:
# ← の ▶ を押して実行してください

import pandas as pd

# 配信一覧 videos 取得
master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
youtube = 'https://youtu.be/'
videos = pd.read_csv(master + 'videos.csv')

# コメント一覧　作成
df_list=[]
for Id in videos['videoId']:
    tmp_df = pd.read_table(master + Id + '.tsv', header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

In [67]:
#df['videoId']= Id

#df2 = df.merge(videos)

videos

,videoId,title,description,publishedAt
0,dPWB8CbsJ-E,【雑談】今日もお疲れ様でした！花音もちょっとくるよ( ᐢ˙꒳​˙ᐢ )【Vtuber】,今日はいろいろあったね( ᐢ˙꒳​˙ᐢ ) まったり振り返りでもしよ！ 花音もちょっと遊びに...,2020-04-04T14:50:56.000Z
1,mUltiyD9aSc,【マイクラ】エンドラ討伐RTA( ᐢ˙꒳​˙ᐢ )【Vtuber】,休日のお昼は～～ まったりエンドラ討伐 ( ᐢ˙꒳​˙ᐢ ) 配信なので見やすいようにおぷ...,2020-04-04T10:08:31.000Z
2,NvMIkTa9SXs,【雀魂】コンビ打ち麻雀🔥本当の&quot;お引き&quot;見せます【楠栞桜視点】,舞元啓介 vs ルイス・キャミーの対決 実は私、お引きの方が向いてるんだよね。 ◇◇◇◇...,2020-04-03T16:40:11.000Z
3,iwmWMSu9llE,【誕生日】今日誕生日なのでうたいます【Vtuber】,たんじょうびなのでうたいます じゅうさんさいになりました. ◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇...,2020-04-01T15:02:23.000Z
4,ULrnUj-PvrY,【あつまれ どうぶつの森】花音とわいわいするだけ【Vtuber】,念願のしおかの( ᐢ˙꒳​˙ᐢ ). ◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇◇ ￣￣￣￣￣￣...,2020-03-29T09:03:09.000Z
5,OnQfDQSyyss,【雀魂】コンビ打ち麻雀🔥【楠栞桜視点】,天開司 vs 楠栞桜の直接対決らしい 所謂「差しウマ」ってやつだね( ᐢ˙꒳​˙ᐢ ) ...,2020-03-26T16:38:19.000Z
6,Xs7YsWWrA5s,【テトリス99】テトリス上手くなりたい！！【視聴者参加型】,テトリス上手いイメージどっからきてんだよ あんま得意じゃないんだよ. ◇◇◇◇◇◇◇◇◇◇◇...,2020-03-25T15:18:58.000Z
7,PyZO36SNLxM,【ましゅまろ雑談】６万人✨ありがとうございます！【Vtuber】,登録者数６万人ありがとうございます！ たくさんのましゅまろもありがとう( ´ •̥ ̫ •̥...,2020-03-23T16:02:35.000Z
8,GmKvmu55lWY,【あつまれ どうぶつの森 / ３日目】カブで一攫千金狙います。【お店オープン編】,３日目！ つぶまめのお店が完成✨ 明日には博物館もできるらしい…！ ◇◇◇◇◇◇◇◇◇◇◇◇...,2020-03-21T10:41:16.000Z
9,qHj-hmquV2g,【雀魂】罰ゲーム：裏ドラ６枚のるまで放送辞められません！！【Vtuber】,２時間ぐらいでクリアする予定ですが、 クリアできなかったら二度と耐久やりません。 （明日の1...,2020-03-20T18:45:23.000Z


In [55]:
trans = str.maketrans({':': '',',': ''})
s = df['time'].str.translate(trans).astype(int)
s = s-40*(abs(s)//100)-2400*(abs(s)//10000)-30
s = ((abs(s)+s)//2).astype(str)

df['URL']= youtube + Id + '?t=' + s + 's'

df.append()

,time,name,comment,URL
0,-0:49,D.Y,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
1,-0:49,太陽の塔,テトリスのロゴに適当に紙貼っておこうw,https://youtu.be/LZVdK5kgSOk?t=0s
2,-0:49,Fumy H,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
3,-0:48,司令_つかされい,しばらくスイッチ配信も厳しいか,https://youtu.be/LZVdK5kgSOk?t=0s
4,-0:48,麻雀ビギナーしょーしょん,めでたい日記念,https://youtu.be/LZVdK5kgSOk?t=0s
...,...,...,...,...
782497,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s
782498,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s
782499,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s
782500,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s


In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# %%timeit
# GoogleDriveのファイルを参照する
from google.colab import drive 
drive.mount('/content/drive')

# 全コメントデータを読み込む　（少し時間がかかります)
# 特定の文字が入っているコメントはバグるのでquotecharで指定して無視
DATA_DIR='/content/drive/My Drive/Colab Notebooks/data/'

# DATA_DIR='data'

files = glob.glob(os.path.join(DATA_DIR,'*.tsv'))
df_list=[]
for file in files:
    tmp_df = pd.read_table(file, header=None, names=['time','name','comment','spchat'],quotechar='"', quoting=3)
    tmp_df['URL'] = os.path.basename(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

#「名前」「コメント」に特定のキーワードを含むコメントを抽出

名前 = " "  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
コメント = ""  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = False  #@param {type:"boolean"} 

再生開始時間 =  -30#@param {type:"integer"}

def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

    
df2 = df[name_condition & comment_condition].copy()

# time, file(videoID) → 再生時間指定URL
# start, end指定URLで、動画の一部だけ切り抜くのもいいかも
s = df2['time'].str.replace(':', '').astype(int)
s = s-40*(abs(s)//100)-2400*(abs(s)//10000)+再生開始時間
s = ((abs(s)+s)//2).astype(str)

df2['URL']='https://youtu.be/'+df2['URL'].str.replace('.tsv','?t=')+s+'s'
pd.set_option('display.max_rows', 10000)

df2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time,name,comment,spchat,URL
1,-0:09,kyo sirogumi,ふるえてきた,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
5,-0:08,me ro,来るぞ！,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
8,-0:07,maphy 1125,待機,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
9,-0:07,moti ancoro,しおちゃんをケーキのサンタさんと入れ替えていいですか？いいよ。ありがと,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
19,-0:05,nigo nigo,初配信だから緊張してるだろうな～,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
...,...,...,...,...,...
901918,57:51,O T,あきらめよう！,NaN,https://youtu.be/alQe0l2uEqQ?t=3441s
901925,57:53,F 7,開けないと帰れないよ,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
901926,57:53,ちゃたろう ねこにゃ,いやいやいやいや,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
901927,57:53,bon bon,かわいい,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
